# Exercise 1
Implement the LSI using SVD. Use the the documents given below as
samples.

__Note__: Use Numpy's linalg package to understand the internals of SVD
1. information extraction systems
2. natural language processing
3. speech signal systems
4. speech processing

Use "speech systems" as the query to find out whether the results match
the answer for Question 5 (Assignment 2).

In [4]:
from gensim import corpora, similarities
from gensim import models

In [5]:
# Read (construct) the Corpus
corpus = [
    "information extraction systems",
    "natural language processing",
    "speech signal systems",
    "speech processing"
]

In [6]:
# Tokenize
corpus_text = [doc.split() for doc in corpus]
corpus_text

[['information', 'extraction', 'systems'],
 ['natural', 'language', 'processing'],
 ['speech', 'signal', 'systems'],
 ['speech', 'processing']]

In [7]:
# Reference: https://radimrehurek.com/gensim/tut2.html
# Create Coprus Dictionary
dictionary = corpora.Dictionary(corpus_text)

In [8]:
# Create Document-Term Matrix (Bag-of-Words)
doc_term_matrix = [dictionary.doc2bow(tokens) for tokens in corpus_text]

In [9]:
doc_term_matrix

[[(0, 1), (1, 1), (2, 1)],
 [(3, 1), (4, 1), (5, 1)],
 [(2, 1), (6, 1), (7, 1)],
 [(5, 1), (7, 1)]]

In [10]:
# Create TF-IDF Matrix - by fit(ting) the model
tfidf = models.TfidfModel(doc_term_matrix)

In [11]:
# Transform (Apply) the model
corpus_tfidf = tfidf[doc_term_matrix]

In [12]:
# Create LSI Transformation
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary)

In [13]:
lsi.print_topics()

[(0,
  '0.606*"speech" + 0.523*"processing" + 0.398*"signal" + 0.256*"systems" + 0.233*"language" + 0.233*"natural" + 0.114*"information" + 0.114*"extraction"'),
 (1,
  '-0.434*"information" + -0.434*"extraction" + -0.379*"systems" + 0.368*"language" + 0.368*"natural" + -0.323*"signal" + 0.321*"processing" + -0.025*"speech"'),
 (2,
  '0.472*"information" + 0.472*"extraction" + 0.400*"natural" + 0.400*"language" + -0.351*"signal" + -0.324*"speech" + 0.060*"systems" + 0.051*"processing"'),
 (3,
  '-0.600*"signal" + 0.437*"processing" + -0.351*"language" + -0.351*"natural" + 0.313*"speech" + -0.214*"systems" + 0.172*"information" + 0.172*"extraction"')]

In [14]:
lsi.show_topics()

[(0,
  '0.606*"speech" + 0.523*"processing" + 0.398*"signal" + 0.256*"systems" + 0.233*"language" + 0.233*"natural" + 0.114*"information" + 0.114*"extraction"'),
 (1,
  '-0.434*"information" + -0.434*"extraction" + -0.379*"systems" + 0.368*"language" + 0.368*"natural" + -0.323*"signal" + 0.321*"processing" + -0.025*"speech"'),
 (2,
  '0.472*"information" + 0.472*"extraction" + 0.400*"natural" + 0.400*"language" + -0.351*"signal" + -0.324*"speech" + 0.060*"systems" + 0.051*"processing"'),
 (3,
  '-0.600*"signal" + 0.437*"processing" + -0.351*"language" + -0.351*"natural" + 0.313*"speech" + -0.214*"systems" + 0.172*"information" + 0.172*"extraction"')]

In [15]:
# Create Query Document
query_doc = "speech systems"
query_vec_bow = dictionary.doc2bow(query_doc.split())

In [16]:
# convert the query to LSI space
query_vec_lsi = lsi[query_vec_bow]
query_vec_lsi

[(0, 0.8622500379254149),
 (1, -0.4031576893878383),
 (2, -0.2639245139336671),
 (3, 0.09848073003635041)]

In [17]:
# transform corpus to LSI space and index it
# Reference: https://radimrehurek.com/gensim/tut3.html
index = similarities.MatrixSimilarity(lsi[corpus_tfidf])

In [18]:
# Obtain similarity against all the documents in the corpus
sims = index[query_vec_lsi]

In [19]:
print(list(enumerate(sims)))

[(0, 0.33579946), (1, 0.0), (2, 0.82253736), (3, 0.7123382)]


# Exercise 2

Try to improve the incomplete code (shown during the demo 2)(Topic
modeling using BBC corpus) from the github

<https://github.com/Ramaseshanr/anlp/blob/master/TopicModeling_LSI.ipynb>

Once completed, post your experience in the discussion forum for the benefit of others

In [31]:
import os
import pathlib
import gensim
from gensim.models import LsiModel
from gensim import models
from gensim import corpora
from gensim.utils import lemmatize
import nltk
nltk.download('wordnet')
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem.snowball import EnglishStemmer # Yuva
from nltk.stem import WordNetLemmatizer # Yuva
from gensim.parsing.preprocessing import remove_stopwords, stem_text
from gensim.parsing.preprocessing import strip_numeric, strip_short,strip_multiple_whitespaces,strip_non_alphanum,strip_punctuation,strip_tags,preprocess_string
import pandas as pd
from gensim import similarities
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt
from pprint import pprint

[nltk_data] Downloading package wordnet to /home/ubuntujs/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [34]:
#read the data
corpus_file_name = pathlib.Path("bbc-text.csv")

if corpus_file_name.exists():
    corpus_dir = corpus_file_name
else:
    corpus_dir = 'https://raw.githubusercontent.com/Ramaseshanr/anlp/master/corpus/bbc-text.csv'
    
df_corpus = pd.read_csv(corpus_dir,names=['category', 'text'])

if not corpus_file_name.exists():
    df_corpus.to_csv(corpus_file_name,index=False,header=False)

In [35]:
corpus = df_corpus['text'].values.tolist()
corpus = corpus[1:]

In [ ]:
my_filter = [
    lambda x: x.lower(), strip_tags, strip_punctuation,
    strip_multiple_whitespaces, strip_numeric,
    remove_stopwords, strip_short, stem_text
]

def preprocessing(corpus):
    stemmer = EnglishStemmer()
    lemmatizer = WordNetLemmatizer()
    stop_words = stopwords.words('english')
    for document in corpus:
        doc = strip_numeric(document)
        doc = strip_multiple_whitespaces(doc)
        doc = remove_stopwords(doc)
        strip_tags(doc)
        doc = strip_short(doc,3)
        #doc = stem_text(doc)
        doc = strip_punctuation(doc)
        tokens = gensim.utils.tokenize(doc, lower=True)
        tokens = [token for token in tokens if token not in stop_words ]
        #tokens = [stemmer.stem(token) for token in tokens]
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
        yield tokens
        #yield gensim.utils.tokenize(doc, lower=True)

def preprocessing2(corpus):
    for document in corpus:
        doc = strip_numeric(document)
        doc = strip_multiple_whitespaces(doc)
        doc = remove_stopwords(doc)
        strip_tags(doc)
        doc = strip_short(doc,3)
        #doc = stem_text(doc)
        doc = strip_punctuation(doc)
        yield gensim.utils.tokenize(doc, lower=True)

#texts = preprocessing(corpus)
#for text in texts:
#    print(list(text))

texts = preprocessing(corpus)
dictionary = corpora.Dictionary(texts)

In [ ]:
for d in tokens_:
    print(d)

In [ ]:
dictionary.save_as_text("corpus_dict.txt")

In [ ]:
corpus_vocab = dictionary.itervalues()
print("Corpus Vocab Size: ", len(corpus_vocab), ", Corpus Dictionary Size: ", len(dictionary.token2id))

with open('corpus_vocab.txt', 'w') as f:
    v = sorted(corpus_vocab, key=len)
    for w in v:
        f.write(w + '\n')

In [3]:
dictionary.filter_extremes(no_below=1, keep_n=12000)
print('After filtering: ', len(dictionary.itervalues()))
dictionary.save_as_text("corpus_dict2.txt")

NameError: name 'dictionary' is not defined

In [ ]:
doc_term_matrix = [dictionary.doc2bow(tokens) for tokens in preprocessing(corpus)]
tfidf = models.TfidfModel(doc_term_matrix)
corpus_tfidf = tfidf[doc_term_matrix]

In [ ]:
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary)  # initialize an LSI transformation

In [ ]:
lsi.projection.k

In [ ]:
lsi.projection.s

In [ ]:
pprint(lsi.print_topics(num_topics=5, num_words=25))